<a href="https://colab.research.google.com/github/hroblesdiez/2022_european_men_handball_championship/blob/main/Handball_European_Championships.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro
This is a project to try to predict the Champion of the 22th Handball European  played in 2022 in Hungary and Slovakia acording to all data from the European Champioships played from 1994 to 2020.

# 1. Install all packages and libraries required

---



In [ ]:
#Install Beautiful Soap
!pip install bs4  

#Import pandas
import pandas as pd

#Import numpy
import numpy as np

#Import requests 
import requests

!pip install lxml

#Import Beautiful Soap
from bs4 import BeautifulSoup


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=422bb2339926ccf65732fb787557e2fda71b627c188a63d8315a1826d21ec337
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2. Extracting data from the wikipedia

---



In [ ]:
#Get all the matches played in the European Handball Championship from 1944 until 2022
years = np.arange(1994,2022, 2)

def get_matches(year, historical=True) :
  if historical: 
    URL = f"https://en.wikipedia.org/wiki/{year}_European_Men%27s_Handball_Championship"
  elif historical == False:
    URL = f"https://web.archive.org/web/20210722162340/https://en.wikipedia.org/wiki/2022_European_Men's_Handball_Championship"
  page = requests.get(URL)
  content = page.text

  soup = BeautifulSoup(content, 'lxml')

  matches = soup.find_all(attrs={"cellspacing": "0"})

  home = []
  score = []
  away = []

  for match in matches :
    tds = match.tr.find_all('td')
    home.append(tds[1].get_text(strip=True))
    score.append(tds[2].get_text(strip=True))
    away.append(tds[3].get_text(strip=True))

  dict_handball = {'Home':home, 'Score':score, 'Away':away}
  df_handball = pd.DataFrame(dict_handball)
  df_handball['Year'] = year
  return df_handball  

#Historical data
handball_championships = [get_matches(year) for year in years]
df_HC = pd.concat(handball_championships, ignore_index=True)
df_HC

,Home,Score,Away,Year
0,Germany,23–24,Belarus,1994
1,Russia,27–20,Romania,1994
2,France,27–25,Croatia,1994
3,Croatia,24–22,Germany,1994
4,Belarus,23–31,Russia,1994
...,...,...,...,...
654,Norway,28–29(ET),Croatia,2020
655,Spain,34–32,Slovenia,2020
656,Germany,29–27,Portugal,2020
657,Slovenia,20–28,Norway,2020


In [ ]:
#As we have no yet the matches for the Championship in 2024, I will take the matches from 2022.
df_draw = get_matches(2022, False)
df_draw

,Home,Score,Away,Year
0,Slovenia,v,North Macedonia,2022
1,Denmark,v,Montenegro,2022
2,North Macedonia,v,Montenegro,2022
3,Slovenia,v,Denmark,2022
4,Montenegro,v,Slovenia,2022
...,...,...,...,...
61,I1,v,II2,2022
62,II1,v,I2,2022
63,I3,v,II3,2022
64,LSF1,v,LSF2,2022


In [ ]:
#Retrieve the tables of the different groups in the Preliminary Round

def get_preliminary_round():
  URL = "https://en.wikipedia.org/wiki/2022_European_Men%27s_Handball_Championship"
  page = requests.get(URL)
  content = page.text

  soup = BeautifulSoup(content, 'lxml')

  groups = soup.find_all('table', class_='wikitable')
  preliminary_round = groups[6:-8:1]

  pos = []
  team = []
  pld = np.zeros((24,), dtype=int)
  w = np.zeros((24,), dtype=int)
  d = np.zeros((24,), dtype=int)
  l = np.zeros((24,), dtype=int)
  gf = np.zeros((24,), dtype=int)
  ga = np.zeros((24,), dtype=int)
  gd = np.zeros((24,), dtype=int)
  pts = np.zeros((24,), dtype=int)

  for group in preliminary_round :
    trs_raw = group.find_all('tr')
    trs = trs_raw[1:]
    for tr in trs :
      pos.append(tr.td.get_text(strip=True))
      team.append(tr.th.a.get_text(strip=True))
      tds = tr.find_all('td')
      

  dict_preliminary_round = {'Pos':pos, 'Team':team, 'Pld':pld, 'W':w, 'D':d, 'L':l, 'GF':gf, 'GA':ga, 'GD':gd, 'Pts':pts }
  df_preliminary_round = pd.DataFrame(dict_preliminary_round)

  return df_preliminary_round

df_preliminary_round = get_preliminary_round()

dict_tables = {'Group A': df_preliminary_round[0:4], 'Group B': df_preliminary_round[4:8], 
               'Group C': df_preliminary_round[8:12], 'Group D': df_preliminary_round[12:16], 
               'Group E': df_preliminary_round[16:20], 'Group F': df_preliminary_round[20:24]
              }
dict_tables['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Denmark,0,0,0,0,0,0,0,0
1,2,Montenegro,0,0,0,0,0,0,0,0
2,3,Slovenia,0,0,0,0,0,0,0,0
3,4,North Macedonia,0,0,0,0,0,0,0,0


In [ ]:
#Retrieve the tables of the different groups in the Main Round
def get_main_round():
  URL = "https://en.wikipedia.org/wiki/2022_European_Men%27s_Handball_Championship"
  page = requests.get(URL)
  content = page.text

  soup = BeautifulSoup(content, 'lxml')

  groups = soup.find_all('table', class_='wikitable')
  main_round = groups[-8:-6]

  pos = []
  team = ['A1', 'B1', 'C1', 'A2', 'B2', 'C2', 'D1', 'E1', 'F1', 'D2', 'E2', 'F2',]
  pld = np.zeros((12,), dtype=int)
  w = np.zeros((12,), dtype=int)
  d = np.zeros((12,), dtype=int)
  l = np.zeros((12,), dtype=int)
  gf = np.zeros((12,), dtype=int)
  ga = np.zeros((12,), dtype=int)
  gd = np.zeros((12,), dtype=int)
  pts = np.zeros((12,), dtype=int)

  for group in main_round :
    trs_raw = group.find_all('tr')
    trs = trs_raw[1:]
    for tr in trs :
      pos.append(tr.td.get_text(strip=True))

  dict_main_round = {'Pos':pos, 'Team':team, 'Pld':pld, 'W':w, 'D':d, 'L':l, 'GF':gf, 'GA':ga, 'GD':gd, 'Pts':pts }
  df_main_round = pd.DataFrame(dict_main_round)

  return df_main_round

df_main_round = get_main_round()
df_main_round

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,A1,0,0,0,0,0,0,0,0
1,2,B1,0,0,0,0,0,0,0,0
2,3,C1,0,0,0,0,0,0,0,0
3,4,A2,0,0,0,0,0,0,0,0
4,5,B2,0,0,0,0,0,0,0,0
5,6,C2,0,0,0,0,0,0,0,0
6,1,D1,0,0,0,0,0,0,0,0
7,2,E1,0,0,0,0,0,0,0,0
8,3,F1,0,0,0,0,0,0,0,0
9,4,D2,0,0,0,0,0,0,0,0


In [ ]:
#Get the rank, total points, matches won, losed, and deuce for each team
URL = "https://en.wikipedia.org/wiki/European_Men%27s_Handball_Championship"
page = requests.get(URL)
content = page.text
soup = BeautifulSoup(content, 'lxml')

rank_table = soup.find_all('table', class_='wikitable sortable')

columns =[]
ths = rank_table[0].tbody.tr.find_all('th')
for th in ths :
  head = th.get_text(strip=True)
  columns.append(head)

rank, team, part, points, gp, w, d, l = [], [], [], [], [], [], [], []
trs = rank_table[0].tbody.find_all('tr')
trs = trs[1:]
for tr in trs:
  tds = tr.find_all('td')
  rank.append(tds[0].get_text(strip=True))
  team.append(tds[1].get_text(strip=True))
  part.append(tds[2].get_text(strip=True))
  points.append(tds[3].get_text(strip=True))
  gp.append(tds[4].get_text(strip=True))
  w.append(tds[5].get_text(strip=True))
  d.append(tds[6].get_text(strip=True))
  l.append(tds[7].get_text(strip=True))

df_rank = pd.DataFrame({'Rank': rank, 'Team': team, 'Participations': part, 'Points': points, 'Games Played': gp, 'Win': w, 'Deuce': d, 'Lost': l})

print(df_rank.isnull().any()) #There are no empty values
df_rank

Rank              False
Team              False
Participations    False
Points            False
Games Played      False
Win               False
Deuce             False
Lost              False
dtype: bool


,Rank,Team,Participations,Points,Games Played,Win,Deuce,Lost
0,1,Spain,15,157,111,75,7,29
1,2,France,15,140,104,65,10,29
2,3,Croatia,15,134,107,62,10,35
3,4,Denmark,14,133,101,63,7,31
4,5,Sweden,14,127,96,60,7,29
5,6,Germany,14,112,97,49,14,34
6,7,Russia,14,93,83,42,9,32
7,8,Iceland,12,68,71,29,10,32
8,9,Norway,10,67,62,30,7,25
9,10,Slovenia,13,63,79,27,9,43


In [ ]:
#Get the medal table

medal_table = soup.find_all('table', class_='wikitable sortable plainrowheaders jquery-tablesorter')
columns =[]
ths = medal_table[0].tbody.tr.find_all('th')
for th in ths :
  head = th.get_text(strip=True)
  columns.append(head)

rank, nation, gold, silver, bronze = [], [], [], [], []
trs = medal_table[0].tbody.find_all('tr')
trs = trs[1:-1]
for tr in trs:
  tds = tr.find_all('td')
  rank.append(tds[0].get_text(strip=True))
  nation.append(tr.th.get_text(strip=True))
  gold.append(tds[1].get_text(strip=True))
  silver.append(tds[2].get_text(strip=True))
  bronze.append(tds[3].get_text(strip=True))
  
df_medal = pd.DataFrame({'Rank': rank, 'Nation': nation, 'Gold': gold, 'Silver': silver, 'Bronze': bronze, })
df_medal['Total'] = df_medal['Gold'].astype('int') + df_medal['Silver'].astype('int') + df_medal['Bronze'].astype('int')
df_medal

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,Sweden,5,1,0,6
1,2,France,3,0,2,5
2,3,Spain,2,5,2,9
3,4,Denmark,2,1,4,7
4,5,Germany,2,1,1,4
5,6,Russia,1,2,0,3
6,7,Croatia,0,3,3,6
7,8,Serbia,0,1,0,1
8,0,Slovenia,1,0,1,2
9,10,FR Yugoslavia,0,0,1,1


In [ ]:
df_medal.describe()

,Total
count,12.000000
mean,4.000000
std,2.593699
min,1.000000
25%,2.000000
50%,3.500000
75%,6.000000
max,9.000000


# 3. Cleaning data extracted

---



In [ ]:
#There are no empty or null values in the HandBall Championship Dataframe
df_HC.isnull().any()

Home     False
Score    False
Away     False
Year     False
dtype: bool

In [ ]:
#Eliminate rows that have the word Final or Semifinal and not the score of a match
rows_to_eliminate = df_HC[(df_HC['Home'] == 'Semifinals') | (df_HC['Home'] == 'Semi-finals') 
                    | (df_HC['Away'] == 'Semifinals') | (df_HC['Away'] == 'Semi-finals') | (df_HC['Home'] == 'Semifinal')]

df_HC.drop(rows_to_eliminate.index, inplace=True)
df_HC

,Home,Score,Away,Year
0,Germany,23–24,Belarus,1994
1,Russia,27–20,Romania,1994
2,France,27–25,Croatia,1994
3,Croatia,24–22,Germany,1994
4,Belarus,23–31,Russia,1994
...,...,...,...,...
654,Norway,28–29(ET),Croatia,2020
655,Spain,34–32,Slovenia,2020
656,Germany,29–27,Portugal,2020
657,Slovenia,20–28,Norway,2020


In [ ]:
#Eliminate blank spaces
df_HC['Home'] = df_HC['Home'].str.strip()
df_HC['Away'] = df_HC['Away'].str.strip()
df_HC['Score'] = df_HC['Score'].str.strip()

In [ ]:
#Find characters not numeric in Score column
df_HC[df_HC['Score'].str.contains('[$\(A-Z)]')]
df_HC['Score'] = df_HC['Score'].str.replace('[$\(A-Z)]', '', regex=True) #replace this not desired characters by empty places
df_HC['Score'] = df_HC['Score'].str.strip()
df_HC #We have no more this (ET) string in the score

,Home,Score,Away,Year
0,Germany,23–24,Belarus,1994
1,Russia,27–20,Romania,1994
2,France,27–25,Croatia,1994
3,Croatia,24–22,Germany,1994
4,Belarus,23–31,Russia,1994
...,...,...,...,...
654,Norway,28–29,Croatia,2020
655,Spain,34–32,Slovenia,2020
656,Germany,29–27,Portugal,2020
657,Slovenia,20–28,Norway,2020


In [ ]:
df_HC[['HomeScore', 'AwayScore']] = df_HC['Score'].str.split('–', expand=True) #Split the score column into two columns
df = df_HC.drop('Score', axis=1) #Remove the Score column
subdata = df.loc[[364,365,366,367,368,369,374,375]] #Rows that haven't been splitted
subdata[['HomeScore' ,'AwayScore']] = subdata['HomeScore'].str.split('−', expand=True) #Split these rows
df.loc[[364,365,366,367,368,369,374,375]] = subdata.loc[[364,365,366,367,368,369,374,375]] #Change thses values in main dataframe (df)
df.loc[[364,365,366,367,368,369,374,375]]

,Home,Away,Year,HomeScore,AwayScore
364,Germany,Poland,2010,25,27
365,Sweden,Slovenia,2010,25,27
366,Slovenia,Germany,2010,34,34
367,Poland,Sweden,2010,27,24
368,Germany,Sweden,2010,30,29
369,Poland,Slovenia,2010,30,30
374,France,Spain,2010,24,24
375,Hungary,Czech Republic,2010,26,33


In [ ]:
df.dtypes #checcking the data types in the dataframe

Home         object
Away         object
Year          int64
HomeScore    object
AwayScore    object
dtype: object

In [ ]:
#Change the data types of the HomeScore and AwayScore to int
df = df.astype({'HomeScore': int, 'AwayScore': int})
df.dtypes

Home         object
Away         object
Year          int64
HomeScore     int64
AwayScore     int64
dtype: object

In [ ]:
#Print the number of matches in each year to see if we have to delete anything else or we have remove too much
for year in years :
  print(year, len(df[df['Year']== year]))

1994 38
1996 38
1998 38
2000 38
2002 50
2004 48
2006 47
2008 47
2010 47
2012 47
2014 47
2016 48
2018 47
2020 65


In [ ]:
#Assign the medals won by the FR Yugoslavia team to the Serbia team  and remove the FR Yugoslavia team

df_medal = df_medal.astype({"Gold": np.int8, "Silver": np.int8, "Bronze": np.int8, "Total": np.int8}).copy()

for column in ['Gold', 'Silver', 'Bronze', 'Total'] :
  df_medal.at[7, column] +=  df_medal.at[9, column]



In [ ]:
df_medal_clean = df_medal.drop([9]).set_index('Nation').drop(columns=['Rank'])
df_medal_clean

,Gold,Silver,Bronze,Total
Nation,,,,
Sweden,5,1,0,6
France,3,0,2,5
Spain,2,5,2,9
Denmark,2,1,4,7
Germany,2,1,1,4
Russia,1,2,0,3
Croatia,0,3,3,6
Serbia,0,1,1,2
Slovenia,1,0,1,2


# 4. Generating the model

---







# 4.1. Calculate the strength of a team

### 4.1.1. Calculate the mean of goals scored and received of each team

In [ ]:
# Get the home and away score columns
df_home = df[['Home', 'HomeScore', 'AwayScore']]
df_away = df[['Away', 'HomeScore', 'AwayScore']] 


In [ ]:
# Rename columns
df_home = df_home.rename(columns={'Home': 'Team', 'HomeScore': 'GoalsScored', 'AwayScore': 'GoalsReceived'})
df_away = df_away.rename(columns={'Away': 'Team', 'HomeScore': 'GoalsReceived', 'AwayScore': 'GoalsScored'})
df_home

,Team,GoalsScored,GoalsReceived
0,Germany,23,24
1,Russia,27,20
2,France,27,25
3,Croatia,24,22
4,Belarus,23,31
...,...,...,...
654,Norway,28,29
655,Spain,34,32
656,Germany,29,27
657,Slovenia,20,28


In [ ]:
# Concatenate both dataframes: Home and Away, group by team and obtain the mean of the scores
df_mean_goals_scored_and_reveived = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
df_mean_goals_scored_and_reveived


,GoalsScored,GoalsReceived
Team,,
Austria,28.545455,30.000000
Belarus,27.470588,30.617647
Bosnia and Herzegovina,24.333333,30.000000
Croatia,26.760000,25.530000
Czech Republic,26.574074,28.629630
Denmark,27.597826,25.750000
FR Yugoslavia,25.400000,24.950000
France,27.726316,25.400000
Germany,26.533333,24.911111


### 4.1.2. Create dataframe with the strength of each team

In [ ]:
df_rank_sorted = df_rank.sort_values(by=["Team"]).copy().set_index('Team').drop(columns=[ 'Rank'])


In [ ]:
df_team_strength = df_rank_sorted.join(df_mean_goals_scored_and_reveived).join(df_medal_clean)
df_team_strength

,Participations,Points,Games Played,Win,Deuce,Lost,GoalsScored,GoalsReceived,Gold,Silver,Bronze,Total
Team,,,,,,,,,,,,
Austria,5,16,25,7,2,16,28.545455,30.000000,NaN,NaN,NaN,NaN
Belarus,7,23,37,11,1,25,27.470588,30.617647,NaN,NaN,NaN,NaN
Bosnia and Herzegovina,2,0,6,0,0,6,24.333333,30.000000,NaN,NaN,NaN,NaN
Croatia,15,134,107,62,10,35,26.760000,25.530000,0.0,3.0,3.0,6.0
Czech Republic,11,39,57,17,5,35,26.574074,28.629630,NaN,NaN,NaN,NaN
Denmark,14,133,101,63,7,31,27.597826,25.750000,2.0,1.0,4.0,7.0
France,15,140,104,65,10,29,27.726316,25.400000,3.0,0.0,2.0,5.0
Germany,14,112,97,49,14,34,26.533333,24.911111,2.0,1.0,1.0,4.0
Hungary,13,51,67,20,11,36,25.484375,27.234375,NaN,NaN,NaN,NaN


In [ ]:
df_team_strength = df_team_strength.drop(columns=['Participations', 'Games Played', 'Win', 'Deuce', 'Lost', 'Gold', 'Silver', 'Bronze']).fillna(0)
df_team_strength

,Points,GoalsScored,GoalsReceived,Total
Team,,,,
Austria,16,28.545455,30.000000,0.0
Belarus,23,27.470588,30.617647,0.0
Bosnia and Herzegovina,0,24.333333,30.000000,0.0
Croatia,134,26.760000,25.530000,6.0
Czech Republic,39,26.574074,28.629630,0.0
Denmark,133,27.597826,25.750000,7.0
France,140,27.726316,25.400000,5.0
Germany,112,26.533333,24.911111,4.0
Hungary,51,25.484375,27.234375,0.0


In [ ]:
# apply normalization techniques

df_team_strength_standarized = df_team_strength.copy().astype('float').rename(columns={'Total': 'Medals'})

for column in df_team_strength_standarized.columns:
    df_team_strength_standarized[column] = (df_team_strength_standarized[column] -
    df_team_strength_standarized[column].mean()) / df_team_strength_standarized[column].std() 
    df_team_strength_standarized[column] = (df_team_strength_standarized[column] -
    df_team_strength_standarized[column].min())
    df_team_strength_standarized[column] = (df_team_strength_standarized[column]*100)/df_team_strength_standarized[column].max()
df_team_strength_standarized['GoalsReceived'] = 100 - df_team_strength_standarized['GoalsReceived'] 
df_team_strength_standarized['Strength'] = df_team_strength_standarized['Points'] + df_team_strength_standarized['GoalsScored'] + df_team_strength_standarized['GoalsReceived'] +df_team_strength_standarized['Medals']
df_team_strength_standarized
  

,Points,GoalsScored,GoalsReceived,Medals,Strength
Team,,,,,
Austria,10.191083,100.000000,32.394366,0.000000,142.585449
Belarus,14.649682,83.149364,24.565037,0.000000,122.364083
Bosnia and Herzegovina,0.000000,33.966746,32.394366,0.000000,66.361112
Croatia,85.350318,72.009501,89.056338,66.666667,313.082824
Czech Republic,24.840764,69.094748,49.765258,0.000000,143.700771
Denmark,84.713376,85.144067,86.267606,77.777778,333.902826
France,89.171975,87.158395,90.704225,55.555556,322.590150
Germany,71.337580,68.456057,96.901408,44.444444,281.139490
Hungary,32.484076,52.011580,67.451585,0.000000,151.947241


### 4.1.3. Fill the tables with the classifications in Preliminary and Main Round

In [ ]:
## Splitting the matches for the Champioship into several dataframes: preliminary-round, main-round, fifth place game, 
## third place game, semifinals and final
df_draw_preliminary_round = df_draw[:36].copy()
df_draw_main_round = df_draw[36:60].copy()
df_draw_fifth_placed = df_draw[63:64].copy()
df_draw_semifinals = df_draw[61:63].copy()
df_draw_third_place = df_draw[64:65].copy()
df_draw_final = df_draw[65:].copy()
df_draw.tail(25)

,Home,Score,Away,Year
41,C1,v,B1,2022
42,B1,v,C2,2022
43,A2,v,C1,2022
44,A1,v,B2,2022
45,B2,v,C2,2022
46,A2,v,B1,2022
47,A1,v,C1,2022
48,D1,v,E1,2022
49,D2,v,F2,2022
50,F1,v,E2,2022


In [ ]:
# Define function to assign points en each match
def assign_points(home,away) :
  points_home, points_away = 0,0
  if (abs(df_team_strength_standarized.loc[home,'Strength'] - df_team_strength_standarized.loc[away,'Strength']) <=2):
    points_home, points_away = 1,1
  elif  df_team_strength_standarized.loc[home,'Strength'] > df_team_strength_standarized.loc[away,'Strength']:
    points_home, points_away = 2,0
  else :
    points_home, points_away = 0,2
  return (points_home, points_away)



In [ ]:
# Run the assig_points function in each match in the Preliminary Round and update tables
for group in dict_tables :
  teams_of_the_group = dict_tables[group]['Team'].values
  df_draw_preliminary_round_group = df_draw_preliminary_round[df_draw_preliminary_round['Home'].isin(teams_of_the_group)]
  for index, row in df_draw_preliminary_round_group.iterrows():
        home, away = row['Home'], row['Away']
        points_home, points_away = assign_points(home, away)
        dict_tables[group].loc[dict_tables[group]['Team'] == home, 'Pts'] += points_home
        dict_tables[group].loc[dict_tables[group]['Team'] == away, 'Pts'] += points_away

  dict_tables[group] = dict_tables[group].sort_values('Pts', ascending=False).reset_index()
  dict_tables[group] = dict_tables[group][['Team', 'Pts']]
 

In [ ]:
dict_tables['Group A']

,Team,Pts
0,Denmark,6
1,Slovenia,4
2,North Macedonia,2
3,Montenegro,0


In [ ]:
group_names = dict_tables.keys()

for group in dict_tables :
  dict_tables[group]['Classification'] = [group[6] +'1', group[6] +'2', 'OUT', 'OUT']

dict_tables['Group F']

,Team,Pts,Classification
0,Russia,6,F1
1,Norway,4,F2
2,Lithuania,2,OUT
3,Slovakia,0,OUT


In [ ]:
#Replace the matches calendar of the main round with the real matches

df_draw_main_round.replace({'A1': dict_tables['Group A'].loc[0,'Team'],
                            'A2': dict_tables['Group A'].loc[1,'Team'],
                            'B1': dict_tables['Group B'].loc[0,'Team'],
                            'B2': dict_tables['Group B'].loc[1,'Team'],
                            'C1': dict_tables['Group C'].loc[0,'Team'],
                            'C2': dict_tables['Group C'].loc[1,'Team'],
                            'D1': dict_tables['Group D'].loc[0,'Team'],
                            'D2': dict_tables['Group D'].loc[1,'Team'],
                            'E1': dict_tables['Group E'].loc[0,'Team'],
                            'E2': dict_tables['Group E'].loc[1,'Team'],
                            'F1': dict_tables['Group F'].loc[0,'Team'],
                            'F2': dict_tables['Group F'].loc[1,'Team'],
                            }, inplace=True)

df_draw_main_round_I = df_draw_main_round[0:12].copy()
df_draw_main_round_II = df_draw_main_round[12:].copy()
df_main_round_table_group_I = df_main_round[0:6].copy()
df_main_round_table_group_II = df_main_round[6:].copy()
df_draw_main_round_I


,Home,Score,Away,Year
36,Denmark,v,Iceland,2022
37,Slovenia,v,Croatia,2022
38,France,v,Hungary,2022
39,Slovenia,v,Hungary,2022
40,Denmark,v,Croatia,2022
41,France,v,Iceland,2022
42,Iceland,v,Croatia,2022
43,Slovenia,v,France,2022
44,Denmark,v,Hungary,2022
45,Hungary,v,Croatia,2022


In [ ]:
#Replace the name of the teams in each Main Round Table
df_main_round_table_group_I.replace(
    {'A1': dict_tables['Group A'].loc[0,'Team'],
     'B1': dict_tables['Group B'].loc[0,'Team'],
     'C1': dict_tables['Group C'].loc[0,'Team'],
     'A2': dict_tables['Group A'].loc[1,'Team'],
     'B2': dict_tables['Group B'].loc[1,'Team'],
     'C2': dict_tables['Group C'].loc[1,'Team'],}, inplace=True)

df_main_round_table_group_II.replace(
    {'D1': dict_tables['Group D'].loc[0,'Team'],
     'E1': dict_tables['Group E'].loc[0,'Team'],
     'F1': dict_tables['Group F'].loc[0,'Team'],
     'D2': dict_tables['Group D'].loc[1,'Team'],
     'E2': dict_tables['Group E'].loc[1,'Team'],
     'F2': dict_tables['Group F'].loc[1,'Team'],}, inplace=True)


In [ ]:
#Eliminate not neccessary columns in each Main Round Table
df_main_round_table_group_I = df_main_round_table_group_I.drop(columns=['Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD'])
df_main_round_table_group_II = df_main_round_table_group_II.drop(columns=['Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD'])


In [ ]:
# Run the assig_points function in each match in the Main Round and update tables
def update_table(df_draw, df_table) :
  for index, row in df_draw.iterrows():
    home, away = row['Home'], row['Away']
    points_home, points_away = assign_points(home, away)
    df_table.loc[df_table['Team'] == home, 'Pts'] += points_home
    df_table.loc[df_table['Team'] == away, 'Pts'] += points_away

  df_table = df_table.sort_values('Pts', ascending=False).reset_index()
  return df_table 

### 4.1.4. Fifth and third place, semifinals and final

In [ ]:
df_MR_table_group_I = update_table(df_draw_main_round_I, df_main_round_table_group_I)
df_MR_table_group_II = update_table(df_draw_main_round_II, df_main_round_table_group_II)

In [ ]:
print('Table Group I', df_MR_table_group_I)


Table Group I    index Pos      Team  Pts
0      0   1   Denmark    8
1      2   3    France    6
2      5   6   Croatia    6
3      1   2   Iceland    2
4      3   4  Slovenia    2
5      4   5   Hungary    0


### 4.1.4.1 Fifth place

In [ ]:
df_draw_fifth_placed.replace({'I3': df_MR_table_group_I.loc[2,'Team'] ,'II3': df_MR_table_group_II.loc[2,'Team']}, inplace=True)

In [ ]:
# Define a function to obtain the winner (not drwan allowed)
def get_winner(home,away) :
  if (df_team_strength_standarized.loc[home,'Strength'] > df_team_strength_standarized.loc[away,'Strength']) :
    winner = home
    looser = away
  else :
    winner = away
    looser = home
  return (winner,looser)

In [ ]:
#Get the winner of the match for the fifth place
winner_fifth_place = get_winner(df_draw_fifth_placed.loc[63,'Home'], df_draw_fifth_placed.loc[63,'Away'])[0]

print('Fifth place:', winner_fifth_place)

Fifth place: Croatia


### 4.1.4.2. Semi-finals

In [ ]:
df_draw_semifinals.replace(
    {'I1': df_MR_table_group_I.loc[0,'Team'], 
     'I2': df_MR_table_group_I.loc[1,'Team'], 
     'II1': df_MR_table_group_II.loc[0,'Team'], 
     'II2': df_MR_table_group_II.loc[1,'Team']
     }, inplace=True)

In [ ]:
#Get the winner each match in semifinals
first_runner_up =  get_winner(df_draw_semifinals.loc[61,'Home'], df_draw_semifinals.loc[61,'Away'])[0]
second_runner_up =  get_winner(df_draw_semifinals.loc[62,'Home'], df_draw_semifinals.loc[62,'Away'])[0]
looser_fs1 =  get_winner(df_draw_semifinals.loc[61,'Home'], df_draw_semifinals.loc[61,'Away'])[1]
looser_fs2 =  get_winner(df_draw_semifinals.loc[62,'Home'], df_draw_semifinals.loc[62,'Away'])[1]
print('First runner up:', first_runner_up)
print('Second runner up:', second_runner_up)

First runner up: Denmark
Second runner up: Spain


### 4.1.4.3. Third place

In [ ]:
#Replace the teams in the match draw for the third place
df_draw_third_place.replace({'LSF1': looser_fs1, 'LSF2': looser_fs2}, inplace=True)

#Get the winner of the third place
winner_bronze_medal = get_winner(df_draw_third_place.loc[64,'Home'], df_draw_third_place.loc[64,'Away'])[0]
winner_fourth_place = get_winner(df_draw_third_place.loc[64,'Home'], df_draw_third_place.loc[64,'Away'])[1]

print('Bronze medal:', winner_bronze_medal)
print('Fourth place:', winner_fourth_place)

Bronze medal: France
Fourth place: Sweden


### 4.1.4.4. Final

In [ ]:
#Replace the teams in the final match draw 
df_draw_final.replace({'WSF1': first_runner_up, 'WSF2': second_runner_up}, inplace=True)

#Get the winner of the final
winner_gold_medal = get_winner(df_draw_final.loc[65,'Home'], df_draw_final.loc[65,'Away'])[0]
winner_silver_medal = get_winner(df_draw_final.loc[65,'Home'], df_draw_final.loc[65,'Away'])[1]
print('Gold medal:', winner_gold_medal)
print('Silver medal:', winner_silver_medal)

Gold medal: Spain
Silver medal: Denmark


#4.2. Prediction Final Classification

In [ ]:
df_prediction_classification = pd.DataFrame(
    {'Rank':[1,2,3,4,5], 
     'Team': [winner_gold_medal, winner_silver_medal, winner_bronze_medal, winner_fourth_place, winner_fifth_place]
     })
df_prediction_classification

,Rank,Team
0,1,Spain
1,2,Denmark
2,3,France
3,4,Sweden
4,5,Croatia


# 4.3. Real Final Classification

In [ ]:
URL = "https://en.wikipedia.org/wiki/2022_European_Men%27s_Handball_Championship"
page = requests.get(URL)
content = page.text
soup = BeautifulSoup(content, 'lxml')
real_classification_table = soup.find_all('table', class_='wikitable')
real_classification_table = real_classification_table[-6:-5]

rank, team  = [], []
trs = real_classification_table[0].tbody.find_all('tr')
trs = trs[2:]
for tr in trs:
  tds = tr.find_all('td')
  rank.append(tds[0].get_text(strip=True))
  team.append(tds[1].get_text(strip=True))
  
df_real_classification = pd.DataFrame({'Rank': rank, 'Team': team })
df_real_classification.loc[[0,1,2],'Rank'] = [1,2,3]
df_real_classification

,Rank,Team
0,1,Sweden
1,2,Spain
2,3,Denmark
3,4,France
4,5,Norway
5,6,Iceland
6,7,Germany
7,8,Croatia
8,9,Russia
9,10,Netherlands
